# New Admin Created Quickly

**Detect rapid creation + privileged group add and roll back.**

This notebook is a **SOC training lab + operational playbook**. It includes:

- Background and objective
- Mock dataset you can run locally (safe, synthetic)
- Splunk SPL, Sentinel KQL, Elastic KQL/EQL examples
- SOAR-style automated response templates
- Exercises + solutions

⚠️ **Safety note**: Response commands here are defensive and should be executed only with proper approval.


## Objective

**Goal:** Investigate and respond to **New Admin Created Quickly**.

**Success criteria:** You can reproduce the detection using the mock dataset and describe containment actions.

**Dataset used:** `iam_events.csv`


## Lab Setup (Run This First)

Load helper functions and the mock dataset.


In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("mock_data")

def load_csv(name: str) -> pd.DataFrame:
    return pd.read_csv(DATA_DIR / name)

def show(df, n=10):
    display(df.head(n))


In [ ]:
df = load_csv('iam_events.csv')
show(df)
print('Rows:', len(df))


## Run the Detection (Mock Data)

This cell demonstrates the detection logic on the synthetic dataset.


In [ ]:
# Demo detection logic
import pandas as pd

# Adjust these filters to test different scenarios
print('Demo logic:', 'User created then added to Administrators/Domain Admins within 30m.')

# Example implementation per dataset:
if 'iam_events.csv' == 'auth_signins.csv':
    # impossible travel / MFA patterns
    user='<USER>'
    d=df.copy()
    if user != '<USER>':
        d=d[d['UserPrincipalName']==user]
    # Sort and show unique locations
    d['TimeGenerated']=pd.to_datetime(d['TimeGenerated'])
    d=d.sort_values('TimeGenerated')
    display(d[['TimeGenerated','UserPrincipalName','IPAddress','Location','ResultType','Roles']])

elif 'iam_events.csv' == 'device_process.csv':
    d=df[df['ProcessCommandLine'].str.contains('-enc|-encodedcommand|FromBase64String', case=False, na=False)]
    display(d)

elif 'iam_events.csv' == 'device_file.csv':
    d=df[df['ActionType'].isin(['FileDeleted','FileRenamed'])]
    display(d)

elif 'iam_events.csv' == 'proxy_logs.csv':
    d=df[df['DestinationHostName'].isin(['drive.google.com','dropbox.com','box.com','onedrive.live.com','mega.nz'])]
    display(d)

elif 'iam_events.csv' == 'dns_events.csv':
    d=df[df['DomainAgeDays']<=30]
    display(d)

elif 'iam_events.csv' == 'office_activity.csv':
    display(df)

elif 'iam_events.csv' == 'iam_events.csv':
    display(df)

elif 'iam_events.csv' == 'vpn_auth.csv':
    d=df[(df['Result']=='Success') & (df['DeviceCompliant']==False)]
    display(d)


## SIEM Queries (Copy/Paste)

Use these in production SIEMs. You will need to adjust field names and table/index names.


In [ ]:
print('### Sentinel KQL (SecurityEvent)\n```kql\nlet window=30m;\nlet newUsers=SecurityEvent | where EventID==4720 | project CreatedTime=TimeGenerated, Target=TargetUserName;\nlet addAdmin=SecurityEvent | where EventID in (4728,4732) | project AddedTime=TimeGenerated, Member=MemberName, Group=TargetUserName;\nnewUsers | join kind=inner (addAdmin) on $left.Target==$right.Member | where AddedTime-CreatedTime < window\n```\n### Splunk SPL\n```spl\nindex=auth (EventCode=4720 OR 4728 OR 4732) | transaction Target_Account maxspan=30m | search EventCode=4720 AND (EventCode=4728 OR 4732)\n```\n### Elastic EQL\n```eql\nsequence with maxspan=30m\n  [ iam where event.action in ("user_created","NewUser") ]\n  [ iam where event.action in ("add_to_group","AddMember") and group.name in ("Domain Admins","Administrators") ]\n```\n')


## Automated Response (SOAR-style)

These cells provide **ready-to-adapt templates** for Sentinel playbooks, Splunk SOAR, and Elastic actions.


In [ ]:
# SOAR-style automation templates (pseudo / ready-to-adapt)

sentinel_logic_app = r'''
# Microsoft Sentinel Playbook (Logic App) - High level steps
# 1) Trigger: When Sentinel alert is created (incident created)
# 2) Actions:
#    - Get entities (user, host, ip, file hash)
#    - Enrich: Risky sign-ins, user info, device risk, TI lookup
#    - Contain: Disable user OR revoke sessions OR isolate device (conditional)
#    - Notify: Teams/Email to SOC channel + ticket creation
#    - Document: Add comments to incident + tags + severity update
'''

splunk_soar = r'''
# Splunk SOAR (Phantom) playbook - High level steps
# - on_start: collect artifacts (user/ip/host/hash)
# - enrich: WHOIS/TI, AD lookup, GeoIP, EDR context
# - decision: if high confidence -> containment actions
# - actions: disable_user, block_ip, isolate_host, add_case_notes
'''

elastic_actions = r'''
# Elastic Security - Rule actions (conceptual)
# - Add action connectors: Email, Slack, Jira, Webhook
# - For high severity: trigger a webhook to SOAR to isolate host / revoke tokens
'''
print("SOAR templates loaded (pseudo/ready-to-adapt).")


## Response Commands (Defensive)

Disable account, remove from groups, investigate creator, rotate privileged creds.

Replace placeholders before executing.


In [ ]:
# Example defensive commands (placeholders)
# Azure AD token revoke:
# Revoke-AzureADUserAllRefreshToken -ObjectId <USER_OBJECT_ID>

# AD disable:
# Disable-ADAccount -Identity <USER>

# MDE isolate device (requires tooling/permissions):
# Invoke-MDATPDeviceIsolation -MachineId <DEVICE_ID>


## Exercises

1) Change `<USER>`/`<HOST>` placeholders and rerun the mock detection.
2) Identify at least two false positive causes.
3) Propose one tuning improvement (allowlist/threshold/correlation).
4) Write a short escalation note (who/what/when/impact/next steps).


## Solutions (Example)

✅ **False positives:** corporate VPN egress, admin maintenance windows, deployment tooling.

✅ **Tuning:** require device mismatch or risky sign-in for impossible travel; exclude jump hosts for SMB bursts; require external domain forward.

✅ **Escalation note template:**
- Summary:
- Entities:
- Evidence:
- Containment:
- Recommended next steps:
